In [1]:
import warnings
warnings.simplefilter(action='ignore')
import nltk
import pandas as pd

In [2]:
train_data = pd.read_csv(r"D:\dev0608\treino.csv", delimiter=';')
test_data = pd.read_csv(r"D:\dev0608\avaliacao.csv", delimiter=';')

In [3]:
train_data

,cod_treino,titulo,categoria,cor,tamanho,marca,tipo,referencia,voltagem,ambiente,subcategoria,linha
0,1,"Piso Laminado Click Nature 136x21,7cm m² Artens",Piso Laminado,Nature,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,NaN,Click
1,2,"Piso Laminado Click Elmo Ravena 136x21,7cm m² ...",Piso Laminado,Ravena,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,NaN,Click
2,3,"Piso Laminado Click Legno Blanche 136x21,7cm m...",Piso Laminado,Blanche,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,NaN,Click
3,4,"Piso Laminado Click Evidence Kalahari 136x21,7...",Piso Laminado,Bege,"136x21,7",Eucafloor,Pisos,NaN,NaN,Interno,NaN,Click
4,5,"Piso Laminado Click Spot Carvalho 134x18,7cm m...",Piso Laminado,Carvalho,"134x18,7",Eucafloor,Pisos,NaN,NaN,Interno,NaN,Click
...,...,...,...,...,...,...,...,...,...,...,...,...
371,14,Guarda-Roupa Casal Ana 6 PT 4 GV Branco 240 cm,Guarda-roupas Casal,Branco,NaN,NaN,Moveis de casal,NaN,NaN,Quarto,NaN,NaN
372,15,Sofá 3 Lugares Living Algodão e Poliéster Cinz...,Sofas,Cinza,NaN,Living,Moveis Estofados,NaN,NaN,Sala & Escritorio,NaN,NaN
373,16,Sofá 4 Lugares Living Poliéster Bege,Sofas,Bege,NaN,Living,Moveis Estofados,NaN,NaN,Sala & Escritorio,NaN,NaN
374,17,Mesa de Jantar Retangular Paraty Amêndoa e Pre...,NaN,Amêndoa & Perola,NaN,NaN,Moveis para cozinha,NaN,NaN,Cozinha,NaN,NaN


In [4]:
train_data.columns

Index(['cod_treino', 'titulo', 'categoria', 'cor', 'tamanho', 'marca', 'tipo',
       'referencia', 'voltagem', 'ambiente', 'subcategoria', 'linha'],
      dtype='object')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download()

In [26]:
import pandas as pd
import spacy
import unidecode
import re
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.pt.stop_words import STOP_WORDS

def preprocess_text(text: str) -> str:
    nlp = spacy.load("pt_core_news_lg")
    text = text.lower()
    text = unidecode.unidecode(text)
    text = re.sub(r'[^a-z\s]', '', text)
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in STOP_WORDS]
    processed_text = " ".join(filtered_tokens)
    return processed_text

def create_embeddings(data: pd.Series) -> pd.Series:
    nlp = spacy.load("pt_core_news_lg")
    embeddings = data.apply(lambda x: nlp(x).vector)
    return embeddings

def fill_category_info(avaliacao_data, treino_data, treino_info):
    similarity_matrix = cosine_similarity(avaliacao_data.tolist(), treino_data.tolist())
    
    for i, row in enumerate(similarity_matrix):
        max_similar_index = row.argmax()
        category_info = treino_info.iloc[max_similar_index]
        yield i, category_info

def main():
    treino = pd.read_csv("treino.csv", delimiter=";")
    avaliacao = pd.read_csv("avaliacao.csv", delimiter=";")
    
    treino['titulo_processed'] = treino['titulo'].apply(preprocess_text)
    avaliacao['titulo_processed'] = avaliacao['titulo'].apply(preprocess_text)
    
    treino_info = treino[['categoria', 'marca', 'subcategoria', 'tipo', 'ambiente']]
    
    treino_embeddings = create_embeddings(treino['titulo_processed'])
    avaliacao_embeddings = create_embeddings(avaliacao['titulo_processed'])
    
    for i, category_info in fill_category_info(avaliacao_embeddings, treino_embeddings, treino_info):
        avaliacao.loc[i, 'categoria'] = category_info['categoria']
        avaliacao.loc[i, 'marca'] = category_info['marca']
        avaliacao.loc[i, 'subcategoria'] = category_info['subcategoria']
        avaliacao.loc[i, 'tipo'] = category_info['tipo']
        avaliacao.loc[i, 'ambiente'] = category_info['ambiente']
    
    print(avaliacao[['titulo', 'categoria', 'marca', 'subcategoria', 'tipo', 'ambiente']])

if __name__ == "__main__":
    main()



                                               titulo            categoria  \
0   Piso Vinílico em Manta Residence Jura Brown Ma...        Piso Vinilico   
1   Piso Vinílico Cola Ambienta Rústico Cinnamon m...        Piso Vinilico   
2   Piso Vinílico Cola Injoy Gerbera Madeira Bege ...        Piso Vinilico   
3   Rodapé Cerâmica Brilhante RHDS1000 15X61cm Artens    Torneira Banheiro   
4   Cuba para Cozinha Dupla de Embutir 82,5x17cm A...             Banheiro   
5   Cuba Inox Pia Cozinha Gourmet Embutir Polido G...             Banheiro   
6   Guarda-Roupa Casal com Espelho Valência 2 PT 6...  Guarda-roupas Casal   
7   Guarda-Roupa Casal Mônaco 6 PT 2 GV Naturale e...  Guarda-roupas Casal   
8   Sofá POLLO 3 Lugares 1,90 metros Linho - 05 Cinza        Piso Vinilico   
9   Sofá Living IDEA 3 Lugares 2,00 metros Linho -...                Sofas   
10   Sofá LION 6 Lugares 3,20 metros Veludo - 02 Bege                Sofas   

        marca         subcategoria              tipo           